In [1]:
#con esto me he conectado
import mysql.connector

mydb = mysql.connector.connect(
  host="dayhoff.inf.um.es",
  user="alubio07",
  password="alubio07PW-",
  database="alubio07"
)

mycursor = mydb.cursor()

In [ ]:
#con esta ejecución creamos la tabla, con el código en sql que se nos ha proporcionado
mycursor.execute("CREATE TABLE genep2 (\
  gene_ensembl varchar(255) NOT NULL,\
  gene_symbol varchar(255) NOT NULL,\
  PRIMARY KEY (gene_ensembl)\
)")

In [ ]:
#lo mismo que en el paso anterior pero con la tabla de proteinp2
mycursor.execute("CREATE TABLE proteinp2 (\
  protein_ensembl varchar(255) NOT NULL,\
  gene_ensembl varchar(255) NOT NULL,\
  sequence longtext,\
  PRIMARY KEY (protein_ensembl),\
  CONSTRAINT fk_protein_1 FOREIGN KEY (gene_ensembl) REFERENCES genep2 (gene_ensembl)\
)")

In [5]:
#con esto comprobamos que aparecen las tablas que acabamos de crear en nuestra base de datos
mycursor.execute("SHOW TABLES")
for x in mycursor:
  print(x)

('departamento',)
('departamento_empleado',)
('empleado',)
('gasto',)
('genep2',)
('manager',)
('presupuesto',)
('proteinp2',)
('salario',)
('sequenceb3',)
('variantb3',)


In [ ]:
#en esta celda he metido el código para insertar el csv en la tabla genep2
import csv

with open("genes.csv", mode='r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=",", )
    next(csv_reader)
    line_count = 0
    values = []
    for row in csv_reader:
        fila = (row[0], row[1])
        values.append(fila) #Imprimimos la columna símbolo

#Creamos la plantilla de inserción

sql_insert = "INSERT INTO genep2 (gene_ensembl, gene_symbol) VALUES (%s, %s)"

mycursor.executemany(sql_insert, values)
mydb.commit()
print(mycursor.rowcount, "filas insertadas")

In [2]:
#aquí he definido una función para obtener del servidos los códigos de ensembl 
#de las proteínas a partir de los códigos ensembl de sus correspondientes genes
def get_protein(gene_ensembl):

    import requests, sys
    server = "https://rest.ensembl.org" #Dirección del ENDPOINT principal

    #Parámetros de la función usando el gene_ensembl y el expand=1

    ext = "/lookup/id/" + gene_ensembl + "?expand=1"

    #Petición GET indicando que quiero que me devuelva en formato JSON

    r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})

 

    #Si la petición es fallida doy error

    if not r.ok:

      raise Exception("Ha fallado")
    
    decoded = r.json()
    #del decoded he mirado donde se encontraba la clase Translation
    #como he visto que estaban dentro de Transcript, he filtrado esa parte
    datos_filtrados=decoded["Transcript"]
    result = []
    for i in range(len(datos_filtrados)):
        #aquí, con el if cojo los que son de la clase "Translation"
        if "Translation" in datos_filtrados[i]:
            #en cada item de la lista se añadirá una tupla con el gen_ensembl (decoded[id])
            #y el protein _ensembl (datos_filtrados[i]["Translation"]["id"]))
            #por lo tanto en distintas tuplas aparecerán repetidos genes,
            #ya que un gen tiene varias proteínas
            result.append((decoded["id"], datos_filtrados[i]["Translation"]["id"]))
    return (result)

In [3]:
#aquí he definido una función para obtener del servidos las secuencias 
#de las proteínas a partir de los códigos ensembl de sus correspondientes proteínas
def get_sequence(protein_ensembl):

    import requests, sys
    server = "https://rest.ensembl.org" #Dirección del ENDPOINT principal

    #Parámetros de la función usando el gen IL6 con CDNA y multiples secuencias

    ext = "/sequence/id/" + protein_ensembl + "?"

    #Petición GET indicando que quiero que me devuelva en formato JSON

    r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})

 

    #Si la petición es fallida doy error

    if not r.ok:

      raise Exception("Ha fallado")
    
    decoded = r.json()
    #el resultado será una tupla con la secuencia y el código de la proteína
    result = (decoded["seq"], decoded["id"])
    return (result)

In [ ]:
#aquí vamos a ejecutar la inserción en la tabla de proteinas
#vamos a tener un error porque ya están insertadas las proteínas

mycursor = mydb.cursor()

mycursor.execute("SELECT * FROM genep2")


myresult = mycursor.fetchall() #Le indicamos que nos cargue todos los valores recuperados

#Creamos la plantilla de inserción

sql_insert = "INSERT INTO proteinp2 (gene_ensembl, protein_ensembl) VALUES (%s, %s)"

for x in myresult:
    protein = get_protein(x[0])
    mycursor.executemany(sql_insert, protein)
    mydb.commit()

In [5]:
#aquí como he puesto un update en lugar de un insert no dará un error, ya que se puede
#hacer update todas las veces que se quiera
mycursor = mydb.cursor()

mycursor.execute("SELECT * FROM proteinp2")


myresult = mycursor.fetchall() #Le indicamos que nos cargue todos los valores recuperados

#Creamos la plantilla de inserción

sql_update = "UPDATE proteinp2 SET sequence = %s WHERE protein_ensembl = %s"

#a pesar de ser código parecido, no he hecho una función de este paso y el anterior
#ya que en el paso de ejecutar se ejecutan cosas distintas
lista = []
for x in myresult:
    sequence = get_sequence(x[0])
    lista.append(sequence)
    mycursor.executemany(sql_update, lista)
    mydb.commit()